In [44]:
import pandas as pd
import requests
import json
import time
import os

In [48]:
# 1. Find all events and event IDs. An event is something like VCT: Masters - Tokyo 2023

events = []

eventsFilePath = "events/events.json"

with open(eventsFilePath, "r") as f:
    events_data = json.load(f)

events = events_data["pageProps"]["events"]

print("Completed Event IDs and Names:")
print("====================")

event_ids = [event["id"] for event in events]

print(event_ids)
print(f"\nTotal completed events found: {len(event_ids)}")

def format_to_csv(df, cols, path):
    df = pd.DataFrame(df)
    df.columns = cols
    df.index = pd.RangeIndex(len(df.index))
    df.to_csv(path)


Completed Event IDs and Names:
[5574, 5441, 5642, 5639, 5632, 5630, 5624, 5638, 5620, 5609, 5614, 5612, 5619, 5627, 5602, 5586, 5575, 5589, 5588, 5621, 5582, 5581, 5577, 5576, 5564, 5605, 5570, 5542, 5546, 5556, 5559, 5549, 5548, 5591, 5535, 5531, 5585, 5598, 5530, 5461, 5476, 5498, 5583, 5469, 5465, 5440, 5437, 5525, 5523, 5528, 5561, 5496, 5484, 5514, 5472, 5489, 5492, 5438, 5474, 5404, 5517, 5463, 5412, 5527, 5453, 5457, 5443, 5450, 5432, 5481, 5387, 5421, 5407, 5459, 5502, 5406, 5397, 5447, 5390, 5386, 5413, 5428, 5426, 5374, 5377, 5436, 5395, 5351, 5368, 5376, 5371, 5365, 5430, 5362, 5383, 5338, 5375, 5359, 5358, 5320, 5333, 5363, 5328, 5347, 5344, 5336, 5329, 5324, 5303, 5316, 5314, 5341, 5310, 5294, 5306, 5297, 5291, 5263, 5232, 5260, 5228, 5224, 5266, 5230, 5279, 5273, 5221, 5248, 5233, 5256, 5405, 5238, 5281, 5244, 5250, 5251, 5216, 5218, 5214, 5252, 5211, 5213, 5201, 5208, 5189, 5205, 5194, 5191, 5197, 5181, 5167, 5164, 5169, 5162, 5179, 5159, 5185, 5175, 5172, 5148, 5050, 51

In [49]:
def fetch_consolidated_match_data(event_ids):
    """Fetch and consolidate kill data with economy and location information"""
    
    all_consolidated_kills = []
    for event_id in event_ids:
        print(f"Processing event ID: {event_id}")
        
        url = f"https://be-prod.rib.gg/v1/series?completed=true&take=50&eventIds%5B%5D={event_id}"
        series_data = requests.get(url).json()
        
        if series_data.get("data") is None:
            print(f"  No data found for event {event_id}")
            continue
        match_ids = []
        map_names = {}
        
        for data in series_data.get("data", []):
            for match in data.get("matches", []):
                match_id = match.get("id")
                if match_id:
                    match_ids.append(match_id)
                    if match.get("map") is not None:
                        map_names[match_id] = match["map"].get("name", "Unknown")
                    else:
                        map_names[match_id] = "Unknown"
        
        print(f"Found {len(match_ids)} matches for event {event_id}")
        
        for match_id in match_ids:
            try:
                print(f"  Processing match ID: {match_id} ({map_names[match_id]})")
                kd_locations_url = f"https://be-prod.rib.gg/v1/x/players/kill-death-locations-by-match?matchId={match_id}"
                try:
                    kd_response = requests.get(kd_locations_url)
                    kd_data = kd_response.json()
                    player_rounds_df = pd.DataFrame(kd_data)
                    print(f"    Got {len(player_rounds_df)} player-round records")
                    
                    player_round_side_map = {}
                    
                    if not player_rounds_df.empty and "playerId" in player_rounds_df.columns and "roundNumber" in player_rounds_df.columns:
                        mapping_df = player_rounds_df[["playerId", "roundNumber", "side"]].dropna()
                        for _, row in mapping_df.iterrows():
                            key = f"{row['playerId']}_{row['roundNumber']}"
                            player_round_side_map[key] = row["side"]
                    
                except Exception as e:
                    print(f"    Error processing kill-death data: {str(e)}")
                    player_round_side_map = {}
                
                match_details_url = f"https://be-prod.rib.gg/v1/matches/{match_id}/details"
                match_details = requests.get(match_details_url).json()
                map_name = map_names[match_id]
                round_mapping = {}
                player_info = {}
                economy_data = {}
                
                for event in match_details.get("events", []):
                    round_id = event.get("roundId")
                    round_num = event.get("roundNumber")
                    if round_id and round_num is not None:
                        round_mapping[round_num] = round_id
                
                for econ in match_details.get("economies", []):
                    round_id = econ.get("roundId")
                    player_id = econ.get("playerId")
                    agent_id = econ.get("agentId")
                    
                    if player_id and agent_id:
                        if player_id not in player_info:
                            player_info[player_id] = {"agent_id": agent_id}
                    
                    if round_id and player_id:
                        key = f"{round_id}_{player_id}"
                        economy_data[key] = econ
                
                kill_events = [e for e in match_details.get("events", []) if e.get("eventType") == "kill"]
                match_kills = []
                
                for kill_event in kill_events:
                    round_id = kill_event.get("roundId")
                    round_number = kill_event.get("roundNumber")
                    round_time = kill_event.get("roundTimeMillis")
                    kill_id = kill_event.get("killId")
                    killer_id = kill_event.get("playerId")
                    victim_id = kill_event.get("referencePlayerId")
                    weapon_id = kill_event.get("weaponId")
                    
                    if not (round_id and killer_id and victim_id):
                        continue
                    
                    killer_economy = economy_data.get(f"{round_id}_{killer_id}", {})
                    victim_economy = economy_data.get(f"{round_id}_{victim_id}", {})
                    
                    killer_locations = []
                    victim_locations = []
                    
                    for loc in match_details.get("locations", []):
                        if loc.get("roundNumber") != round_number:
                            continue
                            
                        time_diff = abs(loc.get("roundTimeMillis", 0) - round_time)
                        
                        if time_diff <= 2000:
                            if loc.get("playerId") == killer_id:
                                killer_locations.append((time_diff, loc))
                            elif loc.get("playerId") == victim_id:
                                victim_locations.append((time_diff, loc))
                    
                    killer_location = min(killer_locations, key=lambda x: x[0])[1] if killer_locations else {}
                    victim_location = min(victim_locations, key=lambda x: x[0])[1] if victim_locations else {}

                    plant_events = [e for e in match_details.get("events", []) 
                                   if e.get("roundId") == round_id and e.get("eventType") == "plant"]
                    plant_time = plant_events[0].get("roundTimeMillis") if plant_events else None

                    plant_site = None
                    if plant_events and len(plant_events) > 0:
                        plant_site = plant_events[0].get("bombsite")

                    killer_side_key = f"{killer_id}_{round_number}"
                    side = player_round_side_map[killer_side_key]
                    
                    consolidated_kill = {
                        # Match/round info
                        "map_name": map_name,
                        "match_id": match_id,
                        "round_id": round_id,
                        "round_number": round_number,
                        "time": round_time,
                        "plant_time": plant_time,
                        "plant_site": plant_site,
                        
                        # Kill info
                        "kill_id": kill_id,
                        "weapon_id": weapon_id,
                        "first_kill": kill_event.get("firstKill", False),
                        "traded_by_kill_id": kill_event.get("tradedByKillId"),
                        "traded_for_kill_id": kill_event.get("tradedForKillId"),
                        
                        # Killer info
                        "killer_id": killer_id,
                        "killer_agent_id": killer_economy.get("agentId"),
                        "killer_weapon_id": killer_economy.get("weaponId"),
                        "killer_armor_id": killer_economy.get("armorId"),
                        "killer_loadout_value": killer_economy.get("loadoutValue"),
                        "killer_x": killer_location.get("locationX"),
                        "killer_y": killer_location.get("locationY"),
                        "killer_view_radians": killer_location.get("viewRadians"),
                        "side": side,
                        
                        # Victim info
                        "victim_id": victim_id,
                        "victim_agent_id": victim_economy.get("agentId"),
                        "victim_weapon_id": victim_economy.get("weaponId"),
                        "victim_armor_id": victim_economy.get("armorId"),
                        "victim_loadout_value": victim_economy.get("loadoutValue"),
                        "victim_x": victim_location.get("locationX"),
                        "victim_y": victim_location.get("locationY"),
                        "victim_view_radians": victim_location.get("viewRadians"),
                        
                        "event_id": event_id,
                    }
                    
                    match_kills.append(consolidated_kill)
                
                all_consolidated_kills.extend(match_kills)
                print(f"    Added {len(match_kills)} kill events from match")
                time.sleep(0.5)  # Be nice to the API
                
            except Exception as e:
                print(f"  Error processing match {match_id}: {str(e)}")
                import traceback
                traceback.print_exc()
    return all_consolidated_kills

In [50]:
if __name__ == "__main__":
    with open("events/events.json", "r") as f:
        events_data = json.load(f)
    
    print(f"Processing {len(event_ids)} events")
    
    consolidated_kills = fetch_consolidated_match_data(event_ids)
    if consolidated_kills:
        df = pd.DataFrame(consolidated_kills)
        
        df.to_csv("datasets/all_kill_data_consolidated.csv", index=False)
        print(f"Saved {len(df)} consolidated kill records")
        
        for map_name, map_group in df.groupby("map_name"):
            map_filename = f"datasets/consolidated_data/{map_name.lower()}_kill_data_consolidated.csv"
            map_group.to_csv(map_filename, index=False)
            print(f"Saved {len(map_group)} kill records for {map_name}")
    else:
        print("No data collected")

Processing 200 events
Processing event ID: 5574
Found 0 matches for event 5574
Processing event ID: 5441
Found 0 matches for event 5441
Processing event ID: 5642
Found 18 matches for event 5642
  Processing match ID: 203244 (Unknown)
    Error processing kill-death data: Expecting value: line 1 column 1 (char 0)
    Added 0 kill events from match
  Processing match ID: 203245 (Unknown)
    Error processing kill-death data: Expecting value: line 1 column 1 (char 0)
    Added 0 kill events from match
  Processing match ID: 203246 (Unknown)
    Error processing kill-death data: Expecting value: line 1 column 1 (char 0)
    Added 0 kill events from match
  Processing match ID: 203235 (Haven)
    Got 0 player-round records
    Added 0 kill events from match
  Processing match ID: 203236 (Pearl)
    Got 0 player-round records
    Added 0 kill events from match
  Processing match ID: 203237 (Unknown)
    Error processing kill-death data: Expecting value: line 1 column 1 (char 0)
    Added 0 k

Traceback (most recent call last):
  File "/tmp/ipykernel_405179/3844560597.py", line 122, in fetch_consolidated_match_data
    side = player_round_side_map[killer_side_key]
KeyError: '25970_1'


    Got 246 player-round records
    Added 123 kill events from match
  Processing match ID: 203181 (Lotus)
    Got 373 player-round records
    Added 189 kill events from match
  Processing match ID: 203182 (Haven)
    Got 194 player-round records
    Added 97 kill events from match
  Processing match ID: 203183 (Ascent)
    Got 478 player-round records
    Added 240 kill events from match
  Processing match ID: 203178 (Lotus)
    Got 254 player-round records
    Added 127 kill events from match
  Processing match ID: 203179 (Pearl)
    Got 348 player-round records
    Added 175 kill events from match
  Processing match ID: 203180 (Sunset)
    Got 238 player-round records
    Added 120 kill events from match
  Processing match ID: 203190 (Haven)
    Got 222 player-round records
    Added 111 kill events from match
  Processing match ID: 203191 (Sunset)
    Got 364 player-round records
    Added 182 kill events from match
  Processing match ID: 203192 (Unknown)
    Error processing kil

Traceback (most recent call last):
  File "/tmp/ipykernel_405179/3844560597.py", line 122, in fetch_consolidated_match_data
    side = player_round_side_map[killer_side_key]
KeyError: '40754_1'


    Got 0 player-round records
  Error processing match 203194: '58914_1'
  Processing match ID: 203195 (Ascent)


...

    Got 237 player-round records
    Added 119 kill events from match
  Processing match ID: 193717 (Fracture)
    Got 321 player-round records
    Added 161 kill events from match
  Processing match ID: 193718 (Pearl)
    Got 317 player-round records
    Added 160 kill events from match
  Processing match ID: 193722 (Icebox)
    Got 349 player-round records
    Added 177 kill events from match
  Processing match ID: 193723 (Pearl)
    Got 224 player-round records
    Added 112 kill events from match
  Processing match ID: 193724 (Lotus)
    Got 284 player-round records
    Added 143 kill events from match
  Processing match ID: 193701 (Lotus)
    Got 216 player-round records
    Added 108 kill events from match
  Processing match ID: 193702 (Haven)
    Got 213 player-round records
    Added 107 kill events from match
  Processing match ID: 193703 (Unknown)
    Error processing kill-death data: Expecting value: line 1 column 1 (char 0)
    Added 0 kill events from match
  Processing ma

Traceback (most recent call last):
  File "/tmp/ipykernel_405179/3844560597.py", line 122, in fetch_consolidated_match_data
    side = player_round_side_map[killer_side_key]
KeyError: '5288_8'


    Got 224 player-round records
    Added 112 kill events from match
  Processing match ID: 186827 (Abyss)
    Got 335 player-round records
    Added 169 kill events from match
  Processing match ID: 186828 (Haven)
    Got 402 player-round records
    Added 201 kill events from match
  Processing match ID: 186829 (Unknown)
    Error processing kill-death data: Expecting value: line 1 column 1 (char 0)
    Added 0 kill events from match
Processing event ID: 5205
Found 154 matches for event 5205
  Processing match ID: 187351 (Ascent)
    Got 262 player-round records
    Added 131 kill events from match
  Processing match ID: 187352 (Haven)
    Got 302 player-round records
    Added 152 kill events from match
  Processing match ID: 187353 (Bind)
    Got 310 player-round records
    Added 155 kill events from match
  Processing match ID: 187374 (Sunset)
    Got 349 player-round records
    Added 175 kill events from match
  Processing match ID: 187375 (Split)
    Got 378 player-round reco

Traceback (most recent call last):
  File "/tmp/ipykernel_405179/3844560597.py", line 122, in fetch_consolidated_match_data
    side = player_round_side_map[killer_side_key]
KeyError: '45401_1'


    Got 289 player-round records
    Added 146 kill events from match
  Processing match ID: 185260 (Unknown)
    Error processing kill-death data: Expecting value: line 1 column 1 (char 0)
    Added 0 kill events from match
  Processing match ID: 185456 (Bind)
    Got 353 player-round records
    Added 178 kill events from match
  Processing match ID: 185457 (Lotus)
    Got 248 player-round records
    Added 124 kill events from match
  Processing match ID: 185458 (Unknown)
    Error processing kill-death data: Expecting value: line 1 column 1 (char 0)
    Added 0 kill events from match
  Processing match ID: 185459 (Icebox)
    Got 182 player-round records
    Added 91 kill events from match
  Processing match ID: 185460 (Haven)
    Got 200 player-round records
    Added 100 kill events from match
  Processing match ID: 185461 (Unknown)
    Error processing kill-death data: Expecting value: line 1 column 1 (char 0)
    Added 0 kill events from match
  Processing match ID: 185453 (Hav